In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import os
!pip install timezonefinder
from timezonefinder import TimezoneFinder
!pip install meteostat
from meteostat import Point, Hourly, Stations
import pytz
from datetime import datetime
from datetime import timedelta

In [ ]:
### Pulling in each team
cols = []
for i in os.listdir('/content/drive/MyDrive/Everything Else/NFL Data/Player CSV/DEF/'):
  team = pd.read_csv('/content/drive/MyDrive/Everything Else/NFL Data/Player CSV/DEF/' + i)
  cols.append(len(team.columns))
print(cols)

In [ ]:
### Formatting each team, and columns within
schedule = pd.DataFrame()
for i in os.listdir('/content/drive/MyDrive/Everything Else/NFL Data/Player CSV/DEF/'):
  team = pd.read_csv('/content/drive/MyDrive/Everything Else/NFL Data/Player CSV/DEF/' + i, header=[0, 1])
  team.columns = [''.join(col) for col in team.columns.values]
  team = team[['Unnamed: 0_level_0Week', 'Unnamed: 1_level_0Day', 'Unnamed: 2_level_0Date', \
               'Unnamed: 3_level_0Unnamed: 3_level_1', 'TmUnnamed: 9_level_1', \
               'Unnamed: 8_level_0Unnamed: 8_level_1', 'Unnamed: 9_level_0Opp', 'YearUnnamed: 27_level_1']]
  team = team[team['Unnamed: 8_level_0Unnamed: 8_level_1'] == '@']
  team = team[team['Unnamed: 0_level_0Week'] != 'Wild Card']
  team = team[team['Unnamed: 0_level_0Week'] != 'Division']
  team = team[team['Unnamed: 0_level_0Week'] != 'Conf. Champ.']
  team = team[team['Unnamed: 0_level_0Week'] != 'SuperBowl']

  team = team.rename(columns={'YearUnnamed: 27_level_1':'Season'})
  team.reset_index(drop=True, inplace=True)
  team['Year'] = ''

  schedule = schedule.append(team)
schedule

In [ ]:
### Continuing to format columns
schedule = schedule.rename(columns = {'Unnamed: 0_level_0Week':'Week', 'Unnamed: 1_level_0Day':'DoW', 'Unnamed: 2_level_0Date':'Dates', 'Unnamed: 3_level_0Unnamed: 3_level_1':'Game_Time_EST', 'TmUnnamed: 9_level_1':'Away Tm',\
                                      'Unnamed: 8_level_0Unnamed: 8_level_1':'@'})
schedule.reset_index(drop=True, inplace=True)
print(schedule.isna().sum())
schedule

In [ ]:
### Formatting for actual year, instead of season
for j in range(len(schedule['Dates'])):
  if 'January' in schedule.loc[j, 'Dates']:
    schedule.loc[j, 'Year'] = schedule.loc[j, 'Season'] + 1
  else:
    schedule.loc[j, 'Year'] = schedule.loc[j, 'Season']

schedule['Date'] = schedule['Dates'] + ', ' + schedule['Year'].astype(str)
schedule['Date'] = pd.to_datetime(schedule['Date'])
schedule

In [ ]:
### Formatting game time
schedule = schedule.sort_values(by=['Dates', 'Game_Time_EST'])
schedule.reset_index(drop=True, inplace=True)
schedule

In [ ]:
### Formatting stadium data
stadium = pd.read_csv('/content/drive/MyDrive/Everything Else/NFL Data/Basic Files/Stadium Data Final.csv')
stadium = stadium.rename(columns = {'Complete Name':'Complete_Name', 'Team Name':'Team_Name', 'Earilest Year':'Earliest_Year', 'Last Year at Stadium':'Last_Year', 'Lat (dec deg':'Lat', 'Long (dec deg':'Long', 'Alt (m)':'Alt'})
stadium

In [ ]:
### Formatting specific lines for unique situations
stadium.loc[54] = [54,	'Washington Redskins', 'Washington',	'Redskins',	'FedEx Field',	'was',	2000,	2019,	79000,	1,	38.907778,	-76.864444,	51,	1,	135.35]
stadium.loc[55] = [55,	'Washington Football Team', 'Washington',	'Football Team',	'FedEx Field',	'was',	2020,	2021,	79000,	1,	38.907778,	-76.864444,	51,	1,	135.35]
stadium.loc[56] = [56,	'Washington Commanders', 'Washington',	'Commanders',	'FedEx Field',	'was',	2022,	2022,	79000,	1,	38.907778,	-76.864444,	51,	1,	135.35]
stadium.loc[26] = [26, 'San Diego Chargers',	'San Diego',	'Chargers',	'SDCCU',	'sdg',	2000,	2016,	71294,	1,	32.783056,	-117.119444,	104,	1,	120.00]
stadium.loc[27] = [27, 'San Diego Chargers',	'San Diego',	'Chargers',	'Dignity Health Sports Park',	'sdg',	2017,	2019,	30000,	1,	33.864000,	-118.261000,	11,	1,	120.00]
for i in range(len(stadium['Last_Year'])):
  if stadium.loc[i, 'Last_Year'] == 2021:
    stadium.loc[i, 'Last_Year'] = 2023
  else:
    continue
stadium

In [51]:
### Adding opponent data
tm_dict = pd.Series(stadium.Tm.values, index=stadium.Complete_Name).to_dict()
tm_dict
schedule['Home_Tm'] = schedule['Unnamed: 9_level_0Opp'].map(tm_dict)

In [ ]:
### Checking for NaN values
schedule = schedule[['DoW', 'Date', 'Game_Time_EST', 'Week', 'Away Tm', '@', 'Home_Tm']]
print(schedule.isna().sum())
schedule

In [ ]:
### Adding new columns for stadium data
schedule['Capacity'] = ''
schedule['Turf'] = ''
schedule['Lat'] = ''
schedule['Long'] = ''
schedule['Alt'] = ''
schedule['In/Out'] = ''
schedule['Dec'] = ''
schedule[1441:1485]

In [ ]:
### Adding stadium data
y = []
for i in range(len(schedule)):
  year = pd.DatetimeIndex(schedule['Date']).year[i]
  team = schedule['Home_Tm'][i]
  x = stadium[(year >= stadium['Earliest Year']) & (year <= stadium['Last_Year']) & (team == stadium['Tm'])]
  x.reset_index(drop=True, inplace=True)
  schedule.loc[i, 'Capacity'] = x._get_value(0, 'Capacity')
  schedule.loc[i, 'Turf'] = x._get_value(0, 'Turf')
  schedule.loc[i, 'Lat'] = x._get_value(0, 'Lat (dec deg)')
  schedule.loc[i, 'Long'] = x._get_value(0, 'Long (dec Deg)')
  schedule.loc[i, 'Alt'] = x._get_value(0, 'Alt')
  schedule.loc[i, 'In/Out'] = x._get_value(0, 'In/Out')
  schedule.loc[i, 'Dec'] = x._get_value(0, 'Dec')

schedule

In [ ]:
### Checking for NaN values per column
schedule[pd.isnull(schedule).any(axis=1)]

In [56]:
### Adding columns for game data and adding data
schedule['TZ'] = ''
schedule['DST'] = ''
obj = TimezoneFinder()
for i in range(len(schedule)):
  tz = obj.timezone_at(lat=schedule['Lat'][i], lng=schedule['Long'][i])
  schedule.loc[i, 'TZ'] = tz

In [ ]:
### Formatting game time data
schedule['Game_Time_EST'] = schedule['Game_Time_EST'].str.replace('ET', '')
schedule['Game_Time_EST'] = schedule['Game_Time_EST'].str.replace('PM', ' PM')
schedule['Game_Time_EST'] = schedule['Game_Time_EST'].str.replace('AM', ' AM')
schedule

In [58]:
### Determining if game location falls into Daylight Savings Time
for i in range(len(schedule)):
  tz = obj.timezone_at(lat=40.812222, lng=-74.076944)
  dst = bool(pytz.timezone(tz).localize(schedule['Date'][i]).dst())
  schedule.loc[i, 'DST'] = dst

In [ ]:
schedule

In [ ]:
### Formatting game time
for i in range(len(schedule['Game_Time_EST'])):
  schedule.loc[i, 'Game_Time_EST'] = schedule.loc[i, 'Game_Time_EST'].strip()
  schedule.loc[i, 'Game_Time_EST'] = datetime.strptime(schedule.loc[i, 'Game_Time_EST'], '%I:%M %p')
  schedule.loc[i, 'Game_Time_EST'] = datetime.strftime(schedule.loc[i, 'Game_Time_EST'], '%H:%M')
  schedule.loc[i, 'Game_Time_EST'] = datetime.strptime(schedule.loc[i, 'Game_Time_EST'], '%H:%M')
schedule

In [ ]:
schedule['TZ'].unique()

In [62]:
### Normalizing EST game time for local game time
schedule['Local_Gametime'] = ''
for i in range(len(schedule)):
  if (schedule.loc[i, 'TZ'] == 'America/Phoenix') and (schedule.loc[i, 'DST'] == True):
    schedule.loc[i, 'Local_Gametime'] = schedule.loc[i, 'Game_Time_EST'] - timedelta(hours=2)
  elif schedule.loc[i, 'TZ'] == 'America/Phoenix' and schedule.loc[i, 'DST'] == False:
    schedule.loc[i, 'Local_Gametime'] = schedule.loc[i, 'Game_Time_EST'] - timedelta(hours=3)
  elif schedule.loc[i, 'TZ'] == 'America/New_York':
    schedule.loc[i, 'Local_Gametime'] = schedule.loc[i, 'Game_Time_EST'] - timedelta(hours=0)
  elif schedule.loc[i, 'TZ'] == 'America/Detroit':
    schedule.loc[i, 'Local_Gametime'] = schedule.loc[i, 'Game_Time_EST'] - timedelta(hours=0)
  elif schedule.loc[i, 'TZ'] == 'America/Chicago':
    schedule.loc[i, 'Local_Gametime'] = schedule.loc[i, 'Game_Time_EST'] - timedelta(hours=1)
  elif schedule.loc[i, 'TZ'] == 'America/Indiana/Indianapolis':
    schedule.loc[i, 'Local_Gametime'] = schedule.loc[i, 'Game_Time_EST'] - timedelta(hours=1)
  elif schedule.loc[i, 'TZ'] == 'America/Denver':
    schedule.loc[i, 'Local_Gametime'] = schedule.loc[i, 'Game_Time_EST'] - timedelta(hours=2)
  elif schedule.loc[i, 'TZ'] == 'America/Los_Angeles':
    schedule.loc[i, 'Local_Gametime'] = schedule.loc[i, 'Game_Time_EST'] - timedelta(hours=3)

In [ ]:
schedule[schedule['Home_Tm'] == 'crd']

In [ ]:
### Formatting local game times
for i in range(len(schedule['Local_Gametime'])):
  schedule.loc[i, 'Local_Gametime'] = schedule.loc[i, 'Local_Gametime'].time().strftime('%H:%M')
schedule

In [ ]:
schedule = schedule[['DoW', 'Date', 'Local_Gametime', 'Week', 'Away Tm', '@', 'Home_Tm', 'Capacity', 'Turf', 'Lat', 'Long', 'Alt', 'In/Out', 'Dec', 'TZ', 'DST']]
schedule

In [ ]:
schedule['Local_Gametime'] = pd.to_datetime(schedule['Local_Gametime']).dt.time
schedule.dtypes

In [ ]:
schedule = schedule.sort_values(by=['Date', 'Local_Gametime'])
schedule.reset_index(drop=True, inplace=True)
schedule

In [ ]:
### Adding columns for weather, and making any in door venue reflect a standard condition
schedule['temp'] = ''
schedule['dwpt'] = ''
schedule['rhum'] = ''
schedule['prcp'] = ''
schedule['snow'] = ''
schedule['wdir'] = ''
schedule['wspd'] = ''
schedule['pres'] = ''

for i in range(len(schedule['Week'])):
  if schedule.loc[i, 'In/Out'] == 0:
    schedule.loc[i, 'temp'] = 23.89
    schedule.loc[i, 'dwpt'] = 15.5
    schedule.loc[i, 'rhum'] = 30
    schedule.loc[i, 'prcp'] = 0
    schedule.loc[i, 'snow'] = 0
    schedule.loc[i, 'wdir'] = 0
    schedule.loc[i, 'wspd'] = 0
    schedule.loc[i, 'pres'] = 1016
schedule

In [ ]:
### Adding weather data.  Green Bay data is sparse for weather stations and data pulls, so we pull the nearest 50 weather stations to average.  
### Everything else we pull in the nearest 10 to average.
### Using the local game time we end 3 hours later for roughly the length of a game.
for i in range(len(schedule['Date'])):
  if schedule.loc[i, 'In/Out'] ==  0:
    continue
  elif schedule.loc[i, 'Home_Tm'] == 'gnb':
    start = datetime(schedule.loc[i, 'Date'].year, schedule.loc[i, 'Date'].month, schedule.loc[i, 'Date'].day, schedule.loc[i, 'Local_Gametime'].hour, schedule.loc[i, 'Local_Gametime'].minute)
    end = start + timedelta(hours=3)
    stations = Stations()
    stations = stations.nearby(schedule.loc[i, 'Lat'], schedule.loc[i, 'Long'])
    stations = stations.fetch(50)
    data = Hourly(stations, start, end)
    data = data.fetch()

    schedule.loc[i, 'temp'] = round(data.mean()['temp'],2)
    schedule.loc[i, 'dwpt'] = round(data.mean()['dwpt'],2)
    schedule.loc[i, 'rhum'] = round(data.mean()['rhum'],2)
    if np.isnan(round(data.mean()['prcp'],2)):
      schedule.loc[i, 'prcp'] = 0
    else:
      schedule.loc[i, 'prcp'] = round(data.mean()['prcp'],2)
    if np.isnan(round(data.mean()['snow'],2)):
      schedule.loc[i, 'snow'] = 0
    else:
      schedule.loc[i, 'snow'] = round(data.mean()['snow'],2)
    schedule.loc[i, 'wdir'] = round(data.mean()['wdir'],2)
    schedule.loc[i, 'wspd'] = round(data.mean()['wspd'],2)
    schedule.loc[i, 'pres'] = round(data.mean()['pres'],2)
  else:
    start = datetime(schedule.loc[i, 'Date'].year, schedule.loc[i, 'Date'].month, schedule.loc[i, 'Date'].day, schedule.loc[i, 'Local_Gametime'].hour, schedule.loc[i, 'Local_Gametime'].minute)
    end = start + timedelta(hours=3)
    stations = Stations()
    stations = stations.nearby(schedule.loc[i, 'Lat'], schedule.loc[i, 'Long'])
    stations = stations.fetch(10)
    data = Hourly(stations, start, end)
    data = data.fetch()

    schedule.loc[i, 'temp'] = round(data.mean()['temp'],2)
    schedule.loc[i, 'dwpt'] = round(data.mean()['dwpt'],2)
    schedule.loc[i, 'rhum'] = round(data.mean()['rhum'],2)
    if np.isnan(round(data.mean()['prcp'],2)):
      schedule.loc[i, 'prcp'] = 0
    else:
      schedule.loc[i, 'prcp'] = round(data.mean()['prcp'],2)
    if np.isnan(round(data.mean()['snow'],2)):
      schedule.loc[i, 'snow'] = 0
    else:
      schedule.loc[i, 'snow'] = round(data.mean()['snow'],2)
    schedule.loc[i, 'wdir'] = round(data.mean()['wdir'],2)
    schedule.loc[i, 'wspd'] = round(data.mean()['wspd'],2)
    schedule.loc[i, 'pres'] = round(data.mean()['pres'],2)

schedule.isna().sum()

In [ ]:
schedule[schedule['temp'].isna()]

In [ ]:
### Certain records are missing, due to the lack of station data.  Adding these manually.
schedule.loc[1, ['temp', 'dwpt', 'rhum', 'prcp', 'snow', 'wdir', 'wspd', 'pres']] = [33.19, 16.18, 40, 0, 0, 90, 3.25, 997.63]
schedule.loc[35, ['temp', 'dwpt', 'rhum', 'prcp', 'snow', 'wdir', 'wspd', 'pres']] = [16.67, 7.77, 50, 0, 0, 270, 10, 995.6]
schedule.loc[73, ['temp', 'dwpt', 'rhum', 'prcp', 'snow', 'wdir', 'wspd', 'pres']] = [8.88, .55, 57, 0, 0, 135, 6, 1000]
schedule.loc[118, ['temp', 'dwpt', 'rhum', 'prcp', 'snow', 'wdir', 'wspd', 'pres']] = [7.22, -3.89, 45, 0, 0, 225, 15, 995.26]
schedule.loc[140, ['temp', 'dwpt', 'rhum', 'prcp', 'snow', 'wdir', 'wspd', 'pres']] = [15, 10.55, 80, 0, 0, 270, 15, 970.2]
schedule.loc[162, ['temp', 'dwpt', 'rhum', 'prcp', 'snow', 'wdir', 'wspd', 'pres']] = [2.78, -6.67, 52, 0, 0, 315, 9, 999]
schedule.loc[210, ['temp', 'dwpt', 'rhum', 'prcp', 'snow', 'wdir', 'wspd', 'pres']] = [7.22, -1.11, 55, 0, 0, 330, 8, 987.13]
schedule.loc[254, ['temp', 'dwpt', 'rhum', 'prcp', 'snow', 'wdir', 'wspd', 'pres']] = [2.22, -1.67, 75, 0, 0, 160, 13, 990.18]
#schedule.loc[585, ['temp', 'dwpt', 'rhum', 'prcp', 'snow', 'wdir', 'wspd', 'pres']] = [6.11, 5, 93, 0, 0, 225, 5, 995.94]
schedule[schedule['wdir'].isna()]

In [ ]:
### In reserve in case this data needs to be changed
#schedule.loc[319, 'wdir'] = [32]
#schedule.loc[86, 'wdir'] = [60]
#schedule.loc[519, 'wdir'] = [300]
#schedule.loc[790, 'wdir'] = [270]

#schedule[schedule['wdir'].isna()]

In [ ]:
### If any wind direction are missing, we fill with 0
schedule['wdir'] = schedule['wdir'].fillna(0)
schedule[schedule['wdir'].isna()]

In [ ]:
schedule.isna().sum()

In [ ]:
### Ensuring week is an int and not object
schedule['Week'] = schedule['Week'].astype('int')
schedule[(schedule['Date'].dt.year == 2022) & (schedule['Week'] == 2)]

In [ ]:
schedule[schedule.temp.isna()]

In [76]:
### Saving the data thus far
schedule.to_csv('/content/drive/MyDrive/Everything Else/NFL Data/Basic Files/Schedule_All.csv')